In [6]:
import os
import time
import requests
from arcgis.gis import GIS
from arcgis.raster import ImageryLayer
import geopandas as gpd
from shapely.geometry import box, Point, Polygon, LineString
import rasterio
from rasterio.mask import mask
from rasterio.merge import merge
from rasterio.io import MemoryFile
import subprocess
import pandas as pd  
import sys
from shapely.ops import unary_union
from datetime import datetime
import shutil
import tempfile

    
def create_tiles(extent, tile_size):
    xmin, ymin, xmax, ymax = extent
    xstep, ystep = tile_size
    tiles = [(x, y, x + xstep, y + ystep) for x in range(int(xmin), int(xmax), xstep) for y in range(int(ymin), int(ymax), ystep)]
    return tiles

def clip_raster_with_geometry(input_raster, geometry, output_raster):
    with rasterio.open(input_raster) as src:
        out_image, out_transform = rasterio.mask.mask(src, [geometry], crop=True)
        out_meta = src.meta.copy()
    
    # Update metadata outside the 'with' block
    out_meta.update({
        "driver": "GTiff",
        "height": out_image.shape[1],
        "width": out_image.shape[2],
        "transform": out_transform
    })
    
    with rasterio.open(output_raster, "w", **out_meta) as dest:
        dest.write(out_image)
    
    print(f"Clipped raster saved: {output_raster}")

    

def process_tile(secure_img_lyr, portal, xmin, ymin, xmax, ymax, i, temp_dir):
    try:
        export_result = secure_img_lyr.export_image(
            bbox=[xmin, ymin, xmax, ymax],
            image_sr=3857,
            size=[1024, 1024],
            export_format='tiff'
        )
        dem_url = export_result['href']
        token = portal._con.token
        dem_url_with_token = f"{dem_url}&token={token}"
        response = requests.get(dem_url_with_token)
        response.raise_for_status()

        memfile = MemoryFile(response.content)
        dataset = memfile.open()
        tile_path = os.path.join(temp_dir, f"Tile_{i}.tif")
        dataset.meta.update({"driver": "GTiff"})
        with rasterio.open(tile_path, 'w', **dataset.meta) as dest:
            dest.write(dataset.read(1), 1)
        print(f"Tile {i + 1} processed successfully")
        return tile_path
    except requests.exceptions.RequestException as e:
        print(f"HTTP request error for Tile {i}: {e}")
    except Exception as e:
        print(f"Error processing DEM for Tile {i}: {e}")
    return None

def generate_and_analyze_contours(dem_path, centroid, output_contour_path, output_max_contour_path):
    contour_interval = 0.1
    try:
        subprocess.run([
            "gdal_contour",
            "-i", str(contour_interval),
            "-a", "ELEV",
            dem_path,
            output_contour_path
        ], check=True, capture_output=True, text=True)
        print(f"Contours generated and saved to {output_contour_path}")
    except subprocess.CalledProcessError as e:
        print("Error generating contours:")
        print(e.stderr)
        print(e.stdout)
        return

    # Load the contours as a GeoDataFrame
    contours_gdf = gpd.read_file(output_contour_path)
    if contours_gdf.crs is None:
        contours_gdf.set_crs(epsg=3857, allow_override=True, inplace=True)

    max_contour = None
    max_elevation = float('-inf')

    # Define a helper function to convert LineString to Polygon
    def convert_to_polygon(geometry):
        if isinstance(geometry, LineString) and geometry.is_ring:
            return Polygon(geometry)
        return None

    # Add a column for polygons
    contours_gdf['polygon'] = contours_gdf.geometry.apply(convert_to_polygon)

    # Add a column for distance to centroid
    contours_gdf['distance_to_centroid'] = contours_gdf.geometry.distance(centroid)

    # Sort the contours by distance to the centroid
    sorted_contours = contours_gdf.sort_values(by='distance_to_centroid')

    # Evaluate contours one by one
    for _, row in sorted_contours.iterrows():
        polygon = row['polygon']
        if polygon and polygon.is_valid:
            if polygon.contains(centroid):
                # Update the max contour if the elevation is higher
                if row['ELEV'] > max_elevation:
                    max_contour = polygon
                    max_elevation = row['ELEV']

    # Save the maximum contour polygon
    if max_contour:
        max_contour_gdf = gpd.GeoDataFrame(
            [{'geometry': max_contour, 'full_suppl': max_elevation}],
            crs='EPSG:3857'
        )
        max_contour_gdf.to_file(output_max_contour_path)
        print(f"Maximum contour polygon saved at {output_max_contour_path}")
    else:
        print("No valid contour found covering the centroid.")
    # **Ensure the GeoDataFrame is fully closed**
    del contours_gdf

    

def combine_max_contour_polygons(output_folder, original_shapefile, LidarZone_project_shapefile):
    max_contour_files = [f for f in os.listdir(output_folder) if f.startswith("MaxContourPolygon_") and f.endswith(".shp")]
    all_max_contours = []

    for contour_file in max_contour_files:
        contour_path = os.path.join(output_folder, contour_file)
        try:
            contour_gdf = gpd.read_file(contour_path)
            all_max_contours.append(contour_gdf)
            print(f"Added {contour_file} to the combined shapefile list.")
        except Exception as e:
            print(f"Error reading {contour_file}: {e}")
    if all_max_contours:
        combined_max_contour_gdf = gpd.GeoDataFrame(pd.concat(all_max_contours, ignore_index=True), crs=all_max_contours[0].crs)

        # Convert to Albers Equal Area projection for area calculation
        combined_max_contour_gdf = combined_max_contour_gdf.to_crs(epsg=3577)

        # Read original shapefile and ensure CRS matches
        try:
            original_gdf = gpd.read_file(original_shapefile)
            original_gdf = original_gdf.to_crs(combined_max_contour_gdf.crs)

            # Perform spatial join to associate PFI values
            combined_max_contour_gdf = gpd.sjoin(combined_max_contour_gdf, original_gdf[['geometry', 'pfi', 'ufi', 'origin', 'function','alternate_', 'qld_pndb_i', 'additional', 'add_names_', 'perenniali', 'hierarchy', 'drainage_b', 'qld_wtr_st', 'constructe', 'volume_ml', 'upper_scal', 'text_note','add_inform', 'feature_ty', 'name', 'attribute_',  'dimension_', 'globalid', 'feature_so']], predicate='intersects')
            # Rename index_right after spatial join to avoid conflict
            if 'index_right' in combined_max_contour_gdf.columns:
                combined_max_contour_gdf.rename(columns={'index_right': 'original_index'}, inplace=True)
            
        except Exception as e:
            print(f"Error loading or joining original shapefile: {e}")
            return

        # Read the lidar_zone shapefile containing lidar project names
        
        try:
            # Read the lidar zone shapefile
            lidar_zone_gdf = gpd.read_file(LidarZone_project_shapefile)
            lidar_zone_gdf = lidar_zone_gdf.to_crs(combined_max_contour_gdf.crs)

            # Rename 'Name' field to 'feature_source'
            lidar_zone_gdf.rename(columns={'Name': 'lidar_source'}, inplace=True)

            # Ensure lidar_zone_gdf is sorted by lidar_year in descending order to prioritize the most recent year
            lidar_zone_gdf = lidar_zone_gdf.sort_values(by='lidar_year', ascending=False)

            # Perform spatial join with lidar zone to get lidar_project name
            combined_max_contour_gdf = gpd.sjoin(combined_max_contour_gdf, lidar_zone_gdf[['geometry', 'lidar_source', 'lidar_year']], predicate='intersects')

            # Drop duplicate feature_sources within the same polygon based on lidar_year (most recent will remain)
            combined_max_contour_gdf = combined_max_contour_gdf.drop_duplicates(subset='geometry', keep='first')

            # Rename the index_right column after the spatial join to avoid conflict
            if 'index_right' in combined_max_contour_gdf.columns:
                combined_max_contour_gdf.rename(columns={'index_right': 'lidar_project_index'}, inplace=True)            

        except Exception as e:
            print(f"Error loading or joining lidar zone shapefile: {e}")


        # Calculate area and add latitude/longitude fields
        combined_max_contour_gdf['Area_sqm'] = combined_max_contour_gdf.geometry.area
        combined_max_contour_gdf['Perimeter_m'] = combined_max_contour_gdf.geometry.boundary.length
        combined_max_contour_gdf['Lon'] = combined_max_contour_gdf.geometry.centroid.x
        combined_max_contour_gdf['Lat'] = combined_max_contour_gdf.geometry.centroid.y
        run_date = datetime.today().strftime('%y-%m-%d')
        combined_max_contour_gdf['run_date'] = run_date

        combined_max_contour_path = os.path.join(output_folder, "Combined_MaxContourPolygons.shp")

        # Rename columns to shorter names for shapefile compatibility
        combined_max_contour_gdf.rename(columns={
            'Area_sqm': 'AreaSqm',
            'Perimeter_m': 'Peri_m',
            'Lon': 'Longitude',
            'Lat': 'Latitude',
            'original_index': 'orig_idx',  # Renamed for compatibility
            'lidar_project_index': 'lidar_proj',
            'lidar_source': 'lidar_so'
            # Renamed for compatibility
        }, inplace=True)

        # Save the combined GeoDataFrame to a shapefile
        if not combined_max_contour_gdf.empty:
            combined_max_contour_gdf.to_file(combined_max_contour_path)
            print(f"All MaxContourPolygons have been combined and saved to {combined_max_contour_path}")
        else:
            print("No MaxContourPolygons found to combine.")
            

def run_combined_script():
    portal = GIS(url='https://auth-spatial.information.qld.gov.au/arcgis/', username='', password='', verify_cert=False)
    secure_url = 'https://auth-spatial-img.information.qld.gov.au/arcgis/rest/services/Elevation/DEM_TimeSeries_CLPUsers/ImageServer'
    secure_img_lyr = ImageryLayer(secure_url, portal)
    output_folder = r'C:\projects\New_Script_SCP\points\outputs_3'
    base_temp_dir = r"C:\projects\temp_storage"
    tile_size = (1000, 1000)
    
    # List to store polygon ID and lidar status
    lidar_status_list = []
    
    shapefile_path = r'C:\projects\New_Script_SCP\points\inputs\point4.shp'
    lidar_coverage_path = r'C:\projects\Lidar_zone\LidarZone_project.shp'

    # Load the shapefile
    try:
        gdf = gpd.read_file(shapefile_path)
        if gdf.empty:
            print("Error: Shapefile is empty.")
            return
    except Exception as e:
        print(f"Error loading shapefile: {e}")
        return

    gdf = gdf.to_crs(epsg=3857)  # Ensure CRS is EPSG:3857

    # Load LiDAR coverage
    try:
        lidar_gdf = gpd.read_file(lidar_coverage_path).to_crs(epsg=3857)
        lidar_union = unary_union(lidar_gdf.geometry)  # Combine all lidar zones into one geometry
        print("Lidar coverage union created successfully")
        lidar_union_gdf = gpd.GeoDataFrame(geometry=[lidar_union], crs=lidar_gdf.crs)
        lidar_union_output_path = os.path.join(output_folder, "Lidar_Union.shp")
        lidar_union_gdf.to_file(lidar_union_output_path)
        print(f"LiDAR union saved to {lidar_union_output_path}")
        
    except Exception as e:
        print(f"Error loading LiDAR coverage: {e}")
        return

    # Process each polygon
    for idx, polygon in gdf.iterrows():
        print(f"Checking water storage {idx + 1}...")

        # Create a temporary directory for each polygon
        temp_dir = tempfile.mkdtemp(dir=base_temp_dir)

        # Extract pfi and ufi values from the shapefile
        pfi = polygon.get('pfi', None)
        ufi = polygon.get('ufi', None)

        # Initialize scp_status to "no" by default
        scp_status = "no"

        # Check if polygon is completely within LiDAR coverage
        if polygon.geometry.within(lidar_union):
            lidar_status = "yes"
            print(f"Polygon {idx + 1} is fully covered by LiDAR.")
        else:
            lidar_status = "no"
            print(f"Polygon {idx + 1} is NOT fully covered by LiDAR.")

        # Append the polygon ID, pfi, ufi, lidar status to the list
        lidar_status_list.append({
            "polygon_id": idx + 1,
            "pfi": pfi,
            "ufi": ufi,
            "lidar_status": lidar_status,
            "scp_status": scp_status  # Add scp_status to the list
        })

        # Continue processing only if fully covered
        if lidar_status == "yes":
            # Check if the geometry is a point or polygon and assign buffer size accordingly
            if isinstance(polygon.geometry, Point):
                buffer_distance = 1000  # Use 1000m buffer for points
                print(f"Point geometry detected for water storage {idx + 1}. Using 5000m buffer.")
            else:
                buffer_distance = 50  # Use 50m buffer for polygons
                print(f"Polygon geometry detected for water storage {idx + 1}. Using 50m buffer.")

            # Create a buffered geometry
            gdf_buffered = polygon.geometry.buffer(buffer_distance)
            buffered_extent = gdf_buffered.bounds
            tiles = create_tiles(buffered_extent, tile_size)
            dem_tiles = []

            for i, (xmin, ymin, xmax, ymax) in enumerate(tiles):
                tile_path = process_tile(secure_img_lyr, portal, xmin, ymin, xmax, ymax, i, temp_dir)
                if tile_path:
                    clipped_tile_path = os.path.join(temp_dir, f"Clipped_Tile_{idx + 1}_{i}.tif")
                    clip_raster_with_geometry(tile_path, gdf_buffered, clipped_tile_path)
                    dem_tiles.append(clipped_tile_path)

            # Open each DEM tile outside the 'with' block to ensure they stay open during the merge
            src_files_to_mosaic = []
            for tile in dem_tiles:
                try:
                    src = rasterio.open(tile)
                    src_files_to_mosaic.append(src)
                except Exception as e:
                    print(f"Error opening tile {tile}: {e}")

            # Now use src_files_to_mosaic to merge
            if src_files_to_mosaic:
                try:
                    mosaic, out_transform = merge(src_files_to_mosaic)
                    mosaic = mosaic.squeeze()

                    # Define the output mosaic path
                    output_mosaic_path = os.path.join(temp_dir, f"Merged_DEM_{idx + 1}.tif")

                    # Write the merged DEM to a file
                    with rasterio.open(output_mosaic_path, 'w', driver='GTiff',
                                       height=mosaic.shape[0], width=mosaic.shape[1], 
                                       count=1, dtype=mosaic.dtype,
                                       crs='EPSG:3857', transform=out_transform) as dst:
                        dst.write(mosaic, 1)

                    print(f"Merged DEM saved to {output_mosaic_path}")
                except Exception as e:
                    print(f"Error during merging tiles: {e}")
                finally:
                    # Ensure files are closed after processing
                    for src in src_files_to_mosaic:
                        src.close()

                    # Generate contours and analyze max contour polygon only if the mosaic was successfully created
                    centroid = polygon.geometry.representative_point()
                    output_contour_path = os.path.join(temp_dir, f"Contours_{idx + 1}.shp")
                    output_max_contour_path = os.path.join(output_folder, f"MaxContourPolygon_{idx + 1}.shp")
                    generate_and_analyze_contours(output_mosaic_path, centroid, output_contour_path, output_max_contour_path)
                    
                try:    
                    # Your main code here
                    if os.path.exists(output_mosaic_path):
                        try:
                            
                            generate_and_analyze_contours(output_mosaic_path, centroid, output_contour_path, output_max_contour_path)

                            # Check if a max contour polygon was found
                            if os.path.exists(output_max_contour_path):
                                scp_status = "yes"  # If max contour polygon exists, update status to 'yes'
                            else:
                                scp_status = "no"  # If not found, keep status as 'no'
                        except Exception as e:
                            print(f"Error during contour generation and analysis: {e}")
                    else:
                        print(f"Mosaic DEM not found for polygon {idx + 1}, skipping contour generation.")
                except Exception as e:
                     print(f"Error during merging or contour analysis for storage {idx + 1}: {e}")
                

            # Delete the intermediate files after processing (after closing them)
                def safe_delete(filepath):
                    try:
                        if os.path.exists(filepath):
                            os.remove(filepath)
                    except Exception as e:
                        print(f"Error deleting {filepath}: {e}")

                # Delete individual tiles
                for tile in dem_tiles:
                    safe_delete(tile)

                # Delete merged DEM
                safe_delete(output_mosaic_path)

                # Delete contour and max contour shapefiles
                safe_delete(output_contour_path)
                

                # Delete temp directory
                try:
                    shutil.rmtree(temp_dir)
                    print(f"Deleted temporary directory: {temp_dir}")
                except Exception as e:
                    print(f"Error deleting {temp_dir}: {e}")
                    
                #except Exception as e:
                    #print(f"Error during merging or contour analysis for storage {idx + 1}: {e}")

        # Update the scp_status in the lidar_status_list
        lidar_status_list[-1]["scp_status"] = scp_status

    # Create DataFrame from lidar_status_list
    lidar_status_df = pd.DataFrame(lidar_status_list)
    
    # Combine all MaxContourPolygons into one shapefile
    combine_max_contour_polygons(
        output_folder=output_folder,
        original_shapefile=shapefile_path,
        LidarZone_project_shapefile=lidar_coverage_path    
    )   

    # Save the updated CSV
    lidar_status_csv_path = os.path.join(output_folder, "lidar_status.csv")
    lidar_status_df.to_csv(lidar_status_csv_path, index=False)
    print(f"LiDAR status with SCP status saved to {lidar_status_csv_path}")

if __name__ == '__main__':
    run_combined_script()

Setting `verify_cert` to False is a security risk, use at your own risk.


Lidar coverage union created successfully
LiDAR union saved to C:\projects\New_Script_SCP\points\outputs_3\Lidar_Union.shp
Checking water storage 1...
Polygon 1 is fully covered by LiDAR.
Point geometry detected for water storage 1. Using 5000m buffer.
Tile 1 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_0.tif
Tile 2 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_1.tif
Tile 3 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_2.tif
Tile 4 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_3.tif
Tile 5 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_4.tif
Tile 6 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7u\Clipped_Tile_1_5.tif
Tile 7 processed successfully
Clipped raster saved: C:\projects\temp_storage\tmp21n2zo7